# Geocoding

In [1]:
import pandas as pd
from opencage.geocoder import OpenCageGeocode
import folium
from folium.plugins import FastMarkerCluster

In [2]:
key = "ce3d9bb0a4cd4f5a9bdb5b1a387fd169"
geocoder = OpenCageGeocode(key)
address='1108 ROSS CLARK CIRCLE,DOTHAN,HOUSTON,AL'
result = geocoder.geocode(address, no_annotations="1")  
result[0]['geometry']

{'lat': 31.2158271, 'lng': -85.3634326}

In [3]:
data=pd.read_csv('Final.csv')
data=data.sample(frac = 0.5)
data.head(10)

,County Name,Facility_Name,Full_Address,State,Total
227,WASHINGTON,VANTAGE POINT OF NORTHWEST ARKANSAS,"4253 CROSSOVER ROAD,FAYETTEVILLE,WASHINGTON,AR",AR,711487
3971,TOM GREEN,RIVER CREST HOSP,"1636 HUNTERS GLEN ROAD,SAN ANGELO,TOM GREEN,TX",TX,22133
1758,PISCATAQUIS,MAYO REGIONAL HOSPITAL,"897 WEST MAIN STREET,DOVER FOXCROFT,PISCATAQUI...",ME,166
1138,ST. CLAIR,TOUCHETTE REGIONAL HOSPITAL INC,"5900 BOND AVENUE,CENTREVILLE,ST. CLAIR,IL",IL,189468
3214,MULTNOMAH,OHSU HOSPITAL AND CLINICS,"3181 SW SAM JACKSON PARK ROAD,PORTLAND,MULTNOM...",OR,148127
4085,ARLINGTON,VIRGINIA HOSPITAL CENTER,"1701 NORTH GEORGE MASON DRIVE,ARLINGTON,ARLING...",VA,186127
448,MADERA,MADERA COMMUNITY HOSPITAL,"1250 E ALMOND AVE,MADERA,MADERA,CA",CA,29128
3303,MONTGOMERY,HOLY REDEEMER HOSPITAL AND MEDICAL CENTER,"1648 HUNTINGDON PIKE,MEADOWBROOK,MONTGOMERY,PA",PA,2265179
1993,DELTA,OSF ST FRANCIS HOSPITAL AND MEDICAL GROUP,"3401 LUDINGTON ST,ESCANABA,DELTA,MI",MI,8630
3022,ASHTABULA,GLENBEIGH HEALTH SOURCES,"2863 STATE ROUTE 45,ROCK CREEK,ASHTABULA,OH",OH,29367


In [4]:
addresses = data["Full_Address"].values.tolist()

In [5]:
key = "ce3d9bb0a4cd4f5a9bdb5b1a387fd169"
geocoder = OpenCageGeocode(key)
latitudes = []
longitudes = []
for address in addresses: 
    result = geocoder.geocode(address, no_annotations="1")  
    
    if result and len(result):  
        longitude = result[0]["geometry"]["lng"]  
        latitude = result[0]["geometry"]["lat"] 
    else:  
        longitude = "N/A"  
        latitude = "N/A"  
    
    latitudes.append(latitude) 
    longitudes.append(longitude)

In [6]:
data["latitudes"] = latitudes
data["longitudes"] = longitudes

In [7]:
data.head(10)

,County Name,Facility_Name,Full_Address,State,Total,latitudes,longitudes
227,WASHINGTON,VANTAGE POINT OF NORTHWEST ARKANSAS,"4253 CROSSOVER ROAD,FAYETTEVILLE,WASHINGTON,AR",AR,711487,37.127182,-82.744495
3971,TOM GREEN,RIVER CREST HOSP,"1636 HUNTERS GLEN ROAD,SAN ANGELO,TOM GREEN,TX",TX,22133,31.445968,-100.511245
1758,PISCATAQUIS,MAYO REGIONAL HOSPITAL,"897 WEST MAIN STREET,DOVER FOXCROFT,PISCATAQUI...",ME,166,45.187688,-69.237320
1138,ST. CLAIR,TOUCHETTE REGIONAL HOSPITAL INC,"5900 BOND AVENUE,CENTREVILLE,ST. CLAIR,IL",IL,189468,38.570024,-90.108429
3214,MULTNOMAH,OHSU HOSPITAL AND CLINICS,"3181 SW SAM JACKSON PARK ROAD,PORTLAND,MULTNOM...",OR,148127,45.500563,-122.688027
4085,ARLINGTON,VIRGINIA HOSPITAL CENTER,"1701 NORTH GEORGE MASON DRIVE,ARLINGTON,ARLING...",VA,186127,38.889780,-77.127554
448,MADERA,MADERA COMMUNITY HOSPITAL,"1250 E ALMOND AVE,MADERA,MADERA,CA",CA,29128,36.944331,-120.044640
3303,MONTGOMERY,HOLY REDEEMER HOSPITAL AND MEDICAL CENTER,"1648 HUNTINGDON PIKE,MEADOWBROOK,MONTGOMERY,PA",PA,2265179,40.110237,-75.082680
1993,DELTA,OSF ST FRANCIS HOSPITAL AND MEDICAL GROUP,"3401 LUDINGTON ST,ESCANABA,DELTA,MI",MI,8630,45.743219,-87.095162
3022,ASHTABULA,GLENBEIGH HEALTH SOURCES,"2863 STATE ROUTE 45,ROCK CREEK,ASHTABULA,OH",OH,29367,41.672153,-80.859367


In [20]:
folium_map= folium.Map(
    location=[33.798259,-84.327062],
    zoom_start=4.4,
    tiles='CartoDB dark_matter')
FastMarkerCluster(data[['latitudes', 'longitudes']].values.tolist()).add_to(folium_map)
folium.LayerControl().add_to(folium_map)

for row in data.iterrows():
    row=row[1]
    folium.CircleMarker(location=(row["latitudes"],
                                  row["longitudes"]),
                        radius= 1,
                        color="#007849",
                        popup=row['Facility_Name'],
                        fill=False).add_to(folium_map)
    
folium_map

# Reverse Geocoding

In [12]:
result = geocoder.reverse_geocode(31.2158271,-85.3634326)  
result[0]['formatted']

'Southeast Health Medical Center, Alma Street, Dothan, AL 36302, United States of America'

In [13]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="test_app")
location = geolocator.reverse("31.2158271,-85.3634326")
location.raw['display_name']

'Southeast Health Campus, 1108, Ross Clark Circle, Morris Heights, Dothan, Houston County, Alabama, 36301, United States of America'

In [14]:
lats=data['latitudes'].to_list()
lons=data['longitudes'].to_list()
coords=list(zip(lats,lons))

In [21]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="test_app")  # enter a name for your app
full_address=[]
for i in range(len(coords)):
    location = geolocator.reverse(coords[i])
    address=location.raw['display_name']
    full_address.append(address)

In [35]:
add=pd.DataFrame(data=full_address , columns=['Address'])
add

,Address
0,"1199, Pine Creek Road, Mayking, Letcher County..."
1,"River Crest Hospital, Hunters Glen Road, The B..."
2,"Mayo Regional Hospital, 897, West Main Street,..."
3,"Touchette Regional Hospital, 5900, Bond Avenue..."
4,"Oregon Health & Science University, Southwest ..."
...,...
2234,"St Luke's Hospital, Anthony Wayne Trail, Maume..."
2235,"Graham Hospital, 210, West Walnut Street, West..."
2236,"599, 2nd South Street, Grace, Caribou County, ..."
2237,"Idaho Falls Community/Mountain View Hospital, ..."
